In [1]:
import torch

from torch.utils.data import Dataset

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device:", device)

PyTorch Version:  1.1.0
Torchvision Version:  0.2.2
Device: cuda:0


In [15]:
data_dir = "data/train/bitmap"

num_epochs = 25


In [4]:
batch_size = 32

print("Initializing Datasets and Dataloaders...")

def bitmap_loader(path):
    with np.load(path) as data:
        return data['arr_0']    

bitmap_dataset = datasets.DatasetFolder(data_dir, loader=bitmap_loader, extensions='npz')

validation_split = 0.2
random_seed = 42

dataset_size = len(bitmap_dataset)
split = int(validation_split * dataset_size)

np.random.seed(random_seed)
indices = np.random.permutation(dataset_size)

train_size = int(0.8 * dataset_size)
val_size = dataset_size - train_size

train_dataset, val_dataset = torch.utils.data.random_split(bitmap_dataset, [train_size, val_size])

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

dataloaders_dict = {}
dataloaders_dict['train'] = train_dataloader
dataloaders_dict['val'] = val_dataloader

Initializing Datasets and Dataloaders...


In [ ]:
for i in range(len(bitmap_dataset)):
    sample = bitmap_dataset[i]
    
    print(i, sample[1])
    
    if i == 100:
        break

In [6]:
bitmap_dataset[3][0].shape

(16, 176400)

In [8]:
class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        
        self.conv_layers = nn.Sequential(
            # input.size: 16x176400
            nn.Conv1d(in_channels=16, out_channels=64, kernel_size=30, stride=10),
            # output: 64 x 17638
            nn.ReLU(),
            nn.BatchNorm1d(64),
            # output 64x17638
            
            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=30, stride=10),
            #output: 256 x 1762
            nn.ReLU(),
            nn.BatchNorm1d(128),
            #output: 256 x 1762
            
            nn.Conv1d(in_channels=128, out_channels=256, kernel_size=30, stride=10),
            #output: 256 x 175
            nn.ReLU(),
            nn.BatchNorm1d(256),
            #output: 256 x 175
        )
        
        self.lstm = nn.LSTM(input_size=256, 
                            hidden_size=512, 
                            num_layers=2, 
                            dropout=0.02)
        
        self.fc = nn.Linear(512, 10)
        
    def forward(self, inputs, hidden):
        output = self.conv_layers(inputs)
        print(output.shape) # should be batch_size x hidden_size x seq_len
        
        output = output.transpose(1,2).transpose(0,1)
        
        output = F.tanh(output)
        output, hidden = self.lstm(output, hidden)
        
        conv_seq_len = output.size(0)
        batch_size = inputs.size(1)
        hidden_size = 512
        output = output.view(conv_seq_len * batch_size, hidden_size)
        output = F.tanh(self.fc(output))
        output = output.view(conv_seq_len, -1, 10)
        return output, hidden
        

In [9]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25):
    since = time.time()
    
    val_acc_history = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-'*10)
        
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            
            running_loss = 0.0
            running_corrects = 0
            
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)
            
        print()
    
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [13]:
model = RNN()

model = model.to(device)

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [16]:
criterion = nn.CrossEntropyLoss()

model_ft, hist = train_model(model, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

Epoch 0/24
----------


TypeError: Traceback (most recent call last):
  File "/home/andre/.local/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 99, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/andre/.local/lib/python3.7/site-packages/torch/utils/data/_utils/collate.py", line 68, in default_collate
    return [default_collate(samples) for samples in transposed]
  File "/home/andre/.local/lib/python3.7/site-packages/torch/utils/data/_utils/collate.py", line 68, in <listcomp>
    return [default_collate(samples) for samples in transposed]
  File "/home/andre/.local/lib/python3.7/site-packages/torch/utils/data/_utils/collate.py", line 52, in default_collate
    return default_collate([torch.from_numpy(b) for b in batch])
  File "/home/andre/.local/lib/python3.7/site-packages/torch/utils/data/_utils/collate.py", line 52, in <listcomp>
    return default_collate([torch.from_numpy(b) for b in batch])
TypeError: can't convert np.ndarray of type nump